# 基礎方程式 Orszag 1971 の (15)-(17) 式
$n$次チェビシェフ多項式$T_n$の$0\leq n\leq N-4$について, 
\begin{equation}
\begin{split}
&\dfrac{1}{24} \sum^{N}_{\substack{p=n+4\\ p\equiv n\; (\mathrm{mod} \; 2)}}{p\left[p^2(p^2-4)^2-3n^2p^4+3n^4p^2-n^2\left(n^2-4 \right) ^2-48\alpha ^2(p^2-n^2) \right] a_p} -8\alpha ^2(n+2)(n+1)a_{n+2} +\alpha ^4c_na_n \\
&\quad -\dfrac{i\alpha R}{2} \left[\sum^{N}_{p=2}{a_p\sum^{}_{\substack{m\equiv p\; (\mathrm{mod} \; 2) \\ |m|\leq p-2 \\ |n-m|\leq N}}{p(p^2-m^2)c_{|n-m|}b_{|n-m|} }} -\alpha ^2\sum^{}_{\substack{|p|\leq N \\ |n-p|\leq N}}{c_{|p|}a_{|p|}c_{|n-p|}b_{|n-p|}} -\sum^{}_{\substack{|p|\leq N \\ |n-p|\leq N}}{c_{|p|}a_{|p|}\sum^{N}_{\substack{m=|n-p|+2\\ m+n\equiv p\; (\mathrm{mod} \; 2)}}{m\left[m^2-(n-p)^2 \right] b_m}} \right] \\
&\quad +\lambda \left\{i\alpha R\sum^{N}_{\substack{p=n+2\\ p\equiv n\; (\mathrm{mod} \; 2)}}{p(p^2-n^2)a_p} -i\alpha ^3Rc_na_n \right\} =0.
\end{split}
 \label{eq:Orszag1971-17-A44}
\end{equation}
および境界条件から得られる係数の関係式:
\begin{align}
a_{N_{\mathrm{e}}}=&\; \sum^{N_{\mathrm{e}}-4}_{\substack{p=0\\ p=0\; (\mathrm{mod}\; 2)}}{\dfrac{(N_{\mathrm{e}}-2)^2-p^2}{4(N_{\mathrm{e}}-1)} a_p} ,\; (\mathrm{even mode}), \label{eq:Orszag1971-17-A45} \\
a_{N_{\mathrm{e}}-2}=&\; \sum^{N_{\mathrm{e}}-4}_{\substack{p=0\\ p=0\; (\mathrm{mod}\; 2)}}{\dfrac{p^2-N^2_{\mathrm{e}}}{4(N_{\mathrm{e}}-1)} a_p} ,\; (\mathrm{even mode}), \label{eq:Orszag1971-17-A46} \\
a_{N_{\mathrm{o}}}=&\; \sum^{N_{\mathrm{o}}-4}_{\substack{p=1\\ p=1\; (\mathrm{mod}\; 2)}}{\dfrac{(N_{\mathrm{o}}-2)^2-p^2}{4(N_{\mathrm{o}}-1)} a_p} ,\; (\mathrm{odd mode}), \label{eq:Orszag1971-17-A47} \\
a_{N_{\mathrm{o}}-2}=&\; \sum^{N_{\mathrm{o}}-4}_{\substack{p=1\\ p=1\; (\mathrm{mod}\; 2)}}{\dfrac{p^2-N^2_{\mathrm{o}}}{4(N_{\mathrm{o}}-1)} a_p} ,\; (\mathrm{odd mode}), \label{eq:Orszag1971-17-A48} 
\end{align}
求める固有値方程式は
$$
(A-B\lambda )\textbf{x} =\textbf{0} \quad \rightarrow \quad \left(B^{-1}A-\lambda I \right) \textbf{x} =\textbf{0},
$$
$$
\textbf{x} \equiv \left[a_0,a_1,\cdots ,a_N \right] ^T.
$$

In [7]:
#using .Lilly1966_solver
using OffsetArrays  # 配列要素番号を任意開始にするモジュール (行列計算時は 1 始まりの通常 matrix に copy する)
using LinearAlgebra

## NOTE: 定式化の添字とコードの配列要素番号は 1 だけオフセットされていることに注意.
##       すなわち, 定式化の a_{n} はコードの a[n+1] を指す.
##       定式化における 2 つの添字 (n,m) の差: n-m はコードでは, (n-1)-(m-1) = n-m

#-- Drawing parameters    
N = 35  # wavenumber in Chebyshev
Ncal = N + 1

# initialize working arrays
### --- From here, supposing matrix element as row, col as in mathematical notation
Acal = reshape(zeros(Complex{Float64},Ncal,Ncal),Ncal,Ncal)  # Matrix for calculation
Bcal = reshape(zeros(Complex{Float64},Ncal,Ncal),Ncal,Ncal)  # Matrix for calculation
A = OffsetArray(Acal,0:N,0:N)  # coefficient matrix
B = OffsetArray(Bcal,0:N,0:N)  # coefficient matrix
### --- From here, supposing matrix element as col, row as in julia notation (only using these arrays for matrix calc)
bcal = reshape(zeros(Ncal),Ncal)  # A vector composed of Chebyshev coefficients for the basic state flow
ccal = reshape(zeros(Ncal),Ncal)  # factorized vector (c_0=2, c_n=1, (n>0))
b = OffsetArray(bcal,0:N)
c = OffsetArray(ccal,0:N)

# setting parameters
α = 1.0
Re = 10000.0

# setting the Chebyshev coefficients for the basic flow
b[0] = 0.5
b[2] = -b[0]

# setting the factorized vector
c[0] = 2.0
c[1:N] .= 1.0

# setting maximum even and odd numbers from N
Nemax = 0
Nomax = 1
for i in 0:2:N
    Nemax = i
end
for i in 1:2:N
    Nomax = i
end
println("Nemax = ", Nemax, ", Nomax = ", Nomax)

# setting matrices
# 支配方程式から構築される部分 (チェビシェフ次数 0<=n<=N-4 の範囲)
# この段階では境界条件で成り立つ係数の関係は考慮しない
for n in 0:N-4
    n2 = n^2
    
    # a_n
    for p in n+4:2:N
        p2 = p^2
        A[n,p] = p * (p2 * (p2 - 4)^2 - 3 * n2 * p2 * (p2 - n2) - n2 * (n2 - 4)^2 - 48 * α^2 * (p2 - n2)) / 24.0
    end
    
    A[n,n+2] = - 8 * α^2 * (n + 2) * (n + 1)
    A[n,n] = α^4 * c[n]
    
    for p in 2:N
        acoef = 0.0
        p2 = p^2
        for m in -(p-2):2:(p-2)
            abs_n_m = abs(n - m)
            m2 = m^2
            if abs_n_m <= N
                acoef = acoef + p * (p2 - m2) * c[abs_n_m] * b[abs_n_m]
            end
        end
        A[n,p] = A[n,p] - complex(0.0,0.5 * α * Re * acoef)
    end

    for p in -N:N
        abs_n_p = abs(n - p)
        abs_p = abs(p)
        if abs_n_p <= N
            A[n,abs_p] = A[n,abs_p] + complex(0.0,0.5 * α^3 * Re * c[abs_p] * c[abs_n_p] * b[abs_n_p])
        end

        if abs_n_p+2 <= N
            acoef = 0.0
            for m in abs_n_p+2:2:N
                m2 = m^2
                acoef = acoef + m * (m2 - (n - p)^2) * b[m]
            end
            A[n,abs_p] = A[n,abs_p] + complex(0.0,0.5 * α * Re * c[abs_p] * acoef)
        end
    end

    # for lambda
    for p in n+2:2:N
        p2 = p^2
        B[n,p] = complex(0.0, - α * Re * p * (p2 - n2))
    end
    B[n,n] = complex(0.0, α^3 * Re * c[n])

end

## a_n (N-3<=n<=N) from boundary conditions
# A(B)[n,N-3:N] に含まれている値 x 境界条件から得られる係数分を A(B)[n,0:N-4] に足し合わせる
for n in 0:N-4  # For each coefficient of T_n (0<=n<=N-4)
    for p in 0:2:Nemax-4  # even mode
        A[n,p] = A[n,p] + (((Nemax - 2)^2 - p^2)/(4 * (Nemax - 1))) * A[n,Nemax]
        A[n,p] = A[n,p] + ((p^2 - Nemax^2)/(4 * (Nemax - 1))) * A[n,Nemax-2]
        B[n,p] = B[n,p] + (((Nemax - 2)^2 - p^2)/(4 * (Nemax - 1))) * B[n,Nemax]
        B[n,p] = B[n,p] + ((p^2 - Nemax^2)/(4 * (Nemax - 1))) * B[n,Nemax-2]
    end
    for p in 1:2:Nomax-4  # odd mode
        A[n,p] = A[n,p] + (((Nomax - 2)^2 - p^2)/(4 * (Nomax - 1))) * A[n,Nomax]
        A[n,p] = A[n,p] + ((p^2 - Nomax^2)/(4 * (Nomax - 1))) * A[n,Nomax-2]
        B[n,p] = B[n,p] + (((Nomax - 2)^2 - p^2)/(4 * (Nomax - 1))) * B[n,Nomax]
        B[n,p] = B[n,p] + ((p^2 - Nomax^2)/(4 * (Nomax - 1))) * B[n,Nomax-2]
    end
end

# copy MATRIX to MATRIXcal
Acal[1:Ncal-4,1:Ncal-4] .= A[0:N-4,0:N-4]
Bcal[1:Ncal-4,1:Ncal-4] .= B[0:N-4,0:N-4]
AAcal = Acal[1:Ncal-4,1:Ncal-4]
BBcal = Bcal[1:Ncal-4,1:Ncal-4]
A_J = transpose(AAcal)
B_J = transpose(BBcal)
Ccal = inv(BBcal) * AAcal
#Ccal = inv(Bcal) * Acal
Dcal = eigvals(Ccal[1:Ncal-4,1:Ncal-4])
F = imag.(Dcal)
Acal[1:Ncal-4,1:Ncal-4]
for i in 1:N-3
    println(Dcal[i])
end
#ncmax = findmax(F)[1]
#Ae = OffsetArray(A[0:div(N,2),0:div(N,2)],0:div(N,2),0:div(N,2))
#Ao = OffsetArray(A[1:div(N,2),1:div(N,2)],1:div(N,2),1:div(N,2))
#Be = OffsetArray(B[0:div(N,2),0:div(N,2)],0:div(N,2),0:div(N,2))
#Bo = OffsetArray(B[1:div(N,2),1:div(N,2)],1:div(N,2),1:div(N,2))
#for n in 0:div(N,2)
#    for p in 0:div(N,2)
#        Ae[n,p] = A[2*n,2*p]
#        Be[n,p] = B[2*n,2*p]
#    end
#end
#for n in 1:div(N,2)
#    for p in 1:div(N,2)
#        Ao[n,p] = A[2*n-1,2*p-1]
#        Bo[n,p] = B[2*n-1,2*p-1]
#    end
#end
#Acal[1:div(N,2)-1,1:div(N,2)-1] .= Ae[0:div(N,2)-2,0:div(N,2)-2]
#Bcal[1:div(N,2)-1,1:div(N,2)-1] .= Be[0:div(N,2)-2,0:div(N,2)-2]
#Ccal = inv(Bcal[1:div(N,2)-1,1:div(N,2)-1]) * Acal[1:div(N,2)-1,1:div(N,2)-1]
##Ccal = inv(Bcal) * Acal
#Dcal = eigvals(Ccal)
#F = imag.(Dcal)
#for i in 1:div(N,2)-1
#    println(F[i])
#end

#Acal[1:div(N,2)-2,1:div(N,2)-2] .= Ao[1:div(N,2)-2,1:div(N,2)-2]
#Bcal[1:div(N,2)-2,1:div(N,2)-2] .= Bo[1:div(N,2)-2,1:div(N,2)-2]
#Ccal = inv(Bcal[1:div(N,2)-2,1:div(N,2)-2]) * Acal[1:div(N,2)-2,1:div(N,2)-2]
##Ccal = inv(Bcal) * Acal
#Dcal = eigvals(Ccal)
#F = imag.(Dcal)
#for i in 1:div(N,2)-2
#    println(F[i])
#end


Nemax = 34, Nomax = 35
0.06094971605335786 + 111.05346004032161im
0.06139187431158177 + 125.37730727421147im
0.06934219291139777 - 4.638847772341679im
0.06937638596763662 - 5.236581226454043im
0.10484135628105651 - 0.9821305816041732im
0.10572983169611849 - 0.8676815968697508im
0.15664131008075105 - 0.3826111349943364im
0.15706670843867665 - 0.3314657508621894im
0.17837652892841122 - 0.1698409856840501im
0.1887331553564245 - 0.19071472954131485im
0.23755870970198328 + 0.00370570625454106im
0.27839043320803625 - 0.05152771511748409im
0.2840686105594082 - 0.1207666508887819im
0.2928747109958421 - 0.12896748508406528im
0.36700268354056875 - 0.08335277702080968im
0.40302859012394104 - 0.08279948220442117im
0.4664156934681825 - 0.0743647989758317im
0.49607616836079294 - 0.07236394251493694im
0.5644681835132881 - 0.06460578106495954im
0.5870367812456175 - 0.06432536046397495im
0.6599597583536706 - 0.055690101118889966im
0.6799353691582228 - 0.055738506933540995im
0.747817317571095 - 0.050016